<a href="https://colab.research.google.com/github/daryl-ramdin/inm707-daryl-ramdin/blob/main/mountaincar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install Packages

In [21]:
# !pip install ray
# !pip install gymnasium
# !pip install 'gymnasium[atari, accept-rom-licesnse]'
# !pip install -U tensorboardx

import ray.rllib.algorithms.dqn as dqn
from ray.rllib.algorithms.ppo import PPOConfig
from ray.tune.logger import pretty_print
from ray import air, tune
import ray
import torch

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [13]:
device = torch.device("cpu")
if torch.cuda.is_available():
    device = torch.device('cuda')

print("Selected device:",device)

Selected device: cpu


In [19]:


ray.shutdown()
ray.init(num_cpus=2)

config = dqn.DEFAULT_CONFIG.copy()

# algo = (
#     PPOConfig()
#     .rollouts(num_rollout_workers=1)
#     .resources(num_gpus=0)
#     .environment(env="CartPole-v1")
#     .build()
# )


# config = PPOConfig().training(lr=tune.grid_search([0.01, 0.001, 0.0001]))

tuner = ray.tune.Tuner(
    "PPO",
    param_space=config,
    run_config=air.RunConfig(
        stop={"episode_reward_mean": 150},
        checkpoint_config=air.CheckpointConfig(checkpoint_at_end=True),
    ),
)
print("Done training")
results = tuner.fit()

# Get the best result based on a particular metric.
best_result = results.get_best_result(metric="episode_reward_mean", mode="max")

# Get the best checkpoint corresponding to the best result.
best_checkpoint = best_result.checkpoint
print("Done")

2023-04-18 23:27:48,619	INFO worker.py:1553 -- Started a local Ray instance.
(pid=6949) /usr/local/lib/python3.9/dist-packages/pkg_resources/__init__.py:121: DeprecationWarning: pkg_resources is deprecated as an API
(pid=6949)   warnings.warn("pkg_resources is deprecated as an API", DeprecationWarning)
(pid=6949) /usr/local/lib/python3.9/dist-packages/pkg_resources/__init__.py:2870: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
(pid=6949) Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
(pid=6949)   declare_namespace(pkg)
(pid=6949) /usr/local/lib/python3.9/dist-packages/pkg_resources/__init__.py:2870: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google.cloud')`.
(pid=6949) Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_r

Done training


2023-04-18 23:29:14,120	WARNING insufficient_resources_manager.py:128 -- Ignore this message if the cluster is autoscaling. You asked for 3.0 cpu and 0 gpu per trial, but the cluster only has 2.0 cpu and 0 gpu. Stop the tuning job and adjust the resources requested per trial (possibly via `resources_per_trial` or via `num_workers` for rllib) and/or add more resources to your Ray runtime.
2023-04-18 23:30:14,244	WARNING insufficient_resources_manager.py:128 -- Ignore this message if the cluster is autoscaling. You asked for 3.0 cpu and 0 gpu per trial, but the cluster only has 2.0 cpu and 0 gpu. Stop the tuning job and adjust the resources requested per trial (possibly via `resources_per_trial` or via `num_workers` for rllib) and/or add more resources to your Ray runtime.
2023-04-18 23:30:26,278	WARNING tune.py:146 -- Stop signal received (e.g. via SIGINT/Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C (or send SIGINT/SIGKILL/SI

KeyboardInterrupt: ignored